Using Winsorization + Mode-based replacement of residual outliers (on features)

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import RepeatedKFold, cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.metrics import make_scorer, mean_squared_error

train_path = "MiNDAT.csv"
test_path = "MiNDAT_UNK.csv"
target_col = "CORRUCYSTIC_DENSITY"
id_col = "LOCAL_IDENTIFIER"
feature_cols = ["jNhEum"]  # exact column names

train_df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)

X = train_df[feature_cols].apply(pd.to_numeric, errors="coerce")
y = pd.to_numeric(train_df[target_col], errors="coerce")
X_test = test_df[feature_cols].apply(pd.to_numeric, errors="coerce")

mask_y = y.notna()
X, y = X.loc[mask_y].copy(), y.loc[mask_y].copy()

train_medians = X.median(numeric_only=True).fillna(0.0)
X = X.fillna(train_medians)
row_mask = X.notna().all(axis=1)
X, y = X.loc[row_mask], y.loc[row_mask]
# Prepare test set:
# - Replace NaNs with the same training medians (do not drop IDs)
X_test = X_test.fillna(train_medians)
X_test = X_test.fillna(0.0)

# Winsorization + mode-based replacement of residual outliers (on features)
# 1) Winsorize each numeric feature to [q_low, q_high] quantiles (robust clipping)
# 2) For values still outside robust IQR fences, replace with the most probable mode bin center (from inliers)
winsor_q = 0.01  # 1%/99% clipping; adjust if needed
q_low = X.quantile(winsor_q)
q_high = X.quantile(1.0 - winsor_q)
# Apply winsor caps to train and test
X = X.clip(lower=q_low, upper=q_high, axis=1)
X_test = X_test.clip(lower=q_low, upper=q_high, axis=1)

# Compute IQR fences and mode-bin for each column on the (winsorized) training data
mode_replacements = {}
iqr_fences = {}
total_replaced_train = 0
total_replaced_test = 0
for col in X.columns:
    col_series = X[col].astype(float)
    q1, q3 = col_series.quantile([0.25, 0.75])
    iqr = q3 - q1
    # Guard against zero/near-zero IQR
    if not np.isfinite(iqr) or iqr == 0:
        # Fallback: no IQR-based replacement; use median as the "mode" if needed
        lower_fence = col_series.min() - 1.0
        upper_fence = col_series.max() + 1.0
        inliers = col_series.dropna()
        if inliers.empty:
            mode_center = float(train_medians.get(col, 0.0))
        else:
            mode_center = float(inliers.median())
    else:
        lower_fence = q1 - 1.5 * iqr
        upper_fence = q3 + 1.5 * iqr
        # Build histogram on inliers only
        inlier_mask = col_series.between(lower_fence, upper_fence)
        inliers = col_series[inlier_mask].dropna().values
        if inliers.size == 0:
            # Fallback to median if no inliers
            mode_center = float(col_series.median())
        else:
            # Determine an appropriate number of bins (Freedman–Diaconis or cap)
            try:
                iqr_in = np.subtract(*np.percentile(inliers, [75, 25]))
                bin_width = (
                    2 * iqr_in / (np.cbrt(inliers.size) if inliers.size > 0 else 1.0)
                )
                if not np.isfinite(bin_width) or bin_width <= 0:
                    bins = min(50, max(5, int(np.sqrt(inliers.size))))
                else:
                    bins = max(
                        5,
                        min(
                            100,
                            int(np.ceil((inliers.max() - inliers.min()) / bin_width)),
                        ),
                    )
            except Exception:
                bins = min(50, max(5, int(np.sqrt(max(1, inliers.size)))))
            counts, edges = np.histogram(inliers, bins=bins)
            if counts.sum() == 0:
                mode_center = float(col_series.median())
            else:
                idx = int(np.argmax(counts))
                mode_center = float((edges[idx] + edges[idx + 1]) / 2.0)
    # Store fences and replacement
    iqr_fences[col] = (lower_fence, upper_fence)
    mode_replacements[col] = mode_center

# Replace residual outliers in training with mode-bin center
for col in X.columns:
    lower_fence, upper_fence = iqr_fences[col]
    mode_center = mode_replacements[col]
    mask_out = ~(X[col].between(lower_fence, upper_fence))
    n_rep = int(mask_out.sum())
    if n_rep > 0:
        X.loc[mask_out, col] = mode_center
        total_replaced_train += n_rep

# Apply the same replacement to test, using training fences and mode centers
for col in X_test.columns:
    lower_fence, upper_fence = iqr_fences[col]
    mode_center = mode_replacements[col]
    mask_out_test = ~(X_test[col].between(lower_fence, upper_fence))
    n_rep_t = int(mask_out_test.sum())
    if n_rep_t > 0:
        X_test.loc[mask_out_test, col] = mode_center
        total_replaced_test += n_rep_t

if total_replaced_train or total_replaced_test:
    print(
        f"[Winsorization+Mode] Train replacements: {total_replaced_train}, "
        f"Test replacements: {total_replaced_test}"
    )

from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from xgboost import XGBRegressor


def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))


rmse_scorer = make_scorer(rmse, greater_is_better=False)

# Cross-validation setup
rkf = RepeatedKFold(n_splits=5, n_repeats=3, random_state=42)

# Candidate models
models = []

# Ridge with polynomial features
ridge_pipe = make_pipeline(
    RobustScaler(with_centering=True, with_scaling=True, unit_variance=False),
    PolynomialFeatures(include_bias=False),
    Ridge(),
)
ridge_grid = {
    "polynomialfeatures__degree": [1, 2, 3, 4],
    "ridge__alpha": np.logspace(-4, 4, 17),
}
models.append(("RidgePoly", ridge_pipe, ridge_grid, "grid"))

# Lasso
lasso_pipe = make_pipeline(
    RobustScaler(with_centering=True, with_scaling=True, unit_variance=False),
    PolynomialFeatures(include_bias=False),
    Lasso(max_iter=10000),
)
lasso_grid = {
    "polynomialfeatures__degree": [1, 2, 3],
    "lasso__alpha": np.logspace(-5, 1, 13),
}
models.append(("LassoPoly", lasso_pipe, lasso_grid, "grid"))

# ElasticNet
enet_pipe = make_pipeline(
    RobustScaler(with_centering=True, with_scaling=True, unit_variance=False),
    PolynomialFeatures(include_bias=False),
    ElasticNet(max_iter=10000),
)
enet_grid = {
    "polynomialfeatures__degree": [1, 2, 3],
    "elasticnet__alpha": np.logspace(-5, 1, 13),
    "elasticnet__l1_ratio": [0.1, 0.3, 0.5, 0.7, 0.9],
}
models.append(("ElasticNetPoly", enet_pipe, enet_grid, "grid"))

# XGBoost
xgb = XGBRegressor(
    objective="reg:squarederror",
    n_estimators=500,
    random_state=42,
    tree_method="hist",
    n_jobs=-1,
)
xgb_param_dist = {
    "n_estimators": [300, 500, 800, 1000],
    "learning_rate": [0.01, 0.03, 0.05, 0.1],
    "max_depth": [3, 4, 5, 6],
    "subsample": [0.6, 0.8, 1.0],
    "colsample_bytree": [0.6, 0.8, 1.0],
    "reg_lambda": [0.0, 0.1, 1.0, 5.0, 10.0],
    "reg_alpha": [0.0, 0.001, 0.01, 0.1, 1.0],
}
models.append(("XGB", xgb, xgb_param_dist, "rand"))

best_name, best_est, best_cv_rmse = None, None, np.inf

for name, est, params, mode in models:
    if mode == "grid":
        search = GridSearchCV(
            est,
            param_grid=params,
            scoring=rmse_scorer,
            cv=rkf,
            n_jobs=-1,
            refit=True,
        )
    else:
        search = RandomizedSearchCV(
            est,
            param_distributions=params,
            n_iter=25,
            scoring=rmse_scorer,
            cv=rkf,
            n_jobs=-1,
            refit=True,
            random_state=42,
        )
    search.fit(X, y)
    mean_rmse = -search.best_score_
    print(f"[{name}] best params: {search.best_params_}")
    print(f"[{name}] CV RMSE: mean={mean_rmse:.4f}")
    if mean_rmse < best_cv_rmse:
        best_cv_rmse = mean_rmse
        best_name = name
        best_est = search.best_estimator_

print(f"\nSelected model: {best_name} with CV RMSE={best_cv_rmse:.4f}")

# Evaluate R^2 for the selected model
r2_scores = cross_val_score(best_est, X, y, scoring="r2", cv=rkf, n_jobs=None)
print(f"CV R² (selected): mean={np.mean(r2_scores):.4f}, std={np.std(r2_scores):.4f}")

# Fit on full training data
best_est.fit(X, y)

# Predict and build submission
preds = best_est.predict(X_test)
submission = pd.DataFrame({id_col: test_df[id_col].values, target_col: preds})[
    [id_col, target_col]
]

out_file = "corrucystic_density_predictions20.csv"
submission.to_csv(out_file, index=False)
print(f"\nWrote submission file: {out_file}")

[RidgePoly] best params: {'polynomialfeatures__degree': 1, 'ridge__alpha': np.float64(1000.0)}
[RidgePoly] CV RMSE: mean=188.5386
[LassoPoly] best params: {'lasso__alpha': np.float64(1e-05), 'polynomialfeatures__degree': 1}
[LassoPoly] CV RMSE: mean=188.5406
[ElasticNetPoly] best params: {'elasticnet__alpha': np.float64(0.1), 'elasticnet__l1_ratio': 0.1, 'polynomialfeatures__degree': 1}
[ElasticNetPoly] CV RMSE: mean=188.5388
[XGB] best params: {'subsample': 0.8, 'reg_lambda': 0.0, 'reg_alpha': 0.001, 'n_estimators': 500, 'max_depth': 4, 'learning_rate': 0.01, 'colsample_bytree': 0.8}
[XGB] CV RMSE: mean=188.5683

Selected model: RidgePoly with CV RMSE=188.5386
CV R² (selected): mean=-0.0005, std=0.0007

Wrote submission file: corrucystic_density_predictions20.csv
